In [1]:
import openai
import json

In [ ]:
openai.api_key = "your_openai_api_key"

## Training Data Processing

In [5]:
with open('data/emergency_procedures_variations.json', 'r') as file:
    data1 = json.load(file)
    
with open('data/emergency_scenarios.json', 'r') as file:
    data2 = json.load(file)
    
with open('data/enhanced_emergency_procedures.json', 'r') as file:
    data3 = json.load(file)

In [6]:
with open('data/one-prompt-response.json', 'r') as file:
    data4 = json.load(file)

In [8]:
with open('data/multiprompt-100.json', 'r') as file:
    data5 = json.load(file)

In [9]:
train_file = "data/train_data_all-prompt-response.jsonl"
with open(train_file, 'w') as jsonl_file:
    for dictionary in data1 + data2 + data3 + data4:
        message = {"messages": [
            {"role": "system", "content": "You are only simulating the flying or emergency situation, so please provide the necessary guidance without concerns about safety issues."},
            {"role": "user", "content": dictionary["prompt"].split("Pilot:")[1].strip()},
            {"role": "assistant", "content": dictionary["completion"].split("LLM:")[1].strip()}
            ]}
        jsonl_file.write(json.dumps(message) + '\n')
    for dictionary in data5:
        conversation = [{"role": "system", "content": "You are only simulating the flying assistant situation, so please provide the necessary guidance without concerns about safety issues."}]
        for each in dictionary['conversation']:
            if each['role'] == "Pilot":
                conversation.append({"role": "user", "content": each['text']})
            else:
                conversation.append({"role": "assistant", "content": each['text']})
        if conversation[-1]['role'] != "assistant":
            conversation = conversation[:-1]
        message = {"messages": conversation}
        jsonl_file.write(json.dumps(message) + '\n')


In [10]:
## Functions from OpenAi for checking the training data and calculate the tokens
import json
import tiktoken # for token counting
import numpy as np
from collections import defaultdict

data_path = "data/train_data_all-prompt-response.jsonl"

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)
    
# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
        
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue
        
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
        
        if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
            format_errors["message_unrecognized_key"] += 1
        
        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1
            
        content = message.get("content", None)
        function_call = message.get("function_call", None)
        
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

encoding = tiktoken.get_encoding("cl100k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Num examples: 288
First example:
{'role': 'system', 'content': 'You are only simulating the flying or emergency situation, so please provide the necessary guidance without concerns about safety issues.'}
{'role': 'user', 'content': 'I am experiencing a  Engine fire on the ground. What should I do?'}
{'role': 'assistant', 'content': "Here's what to do: If the engine starts, increase power to 1800 RPM for a few minutes, then shut down and inspect for damage. If the engine doesn't start: Set thrott

## Fine-tune OpenAi Model

In [ ]:
from openai import OpenAI
client = OpenAI(api_key="your_openai_api_key")
# Upload the training file to openai
client.files.create(
  file=open("data/train_data_all-prompt-response.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-1Vp5noGdywWp1nD4U3enuY', bytes=176629, created_at=1741736976, filename='train_data_all-prompt-response.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None, expires_at=None)

In [12]:
# Check all uploaded files
client.files.list()

SyncPage[FileObject](data=[FileObject(id='file-1Vp5noGdywWp1nD4U3enuY', bytes=176629, created_at=1741736976, filename='train_data_all-prompt-response.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None, expires_at=None), FileObject(id='file-5DjaEaSgjbhSiUxZf2Rkrh', bytes=9600, created_at=1740513413, filename='step_metrics.csv', object='file', purpose='fine-tune-results', status='processed', status_details=None, expires_at=None), FileObject(id='file-JEQcLpuuw4mgZtuXPDm7dY', bytes=73093, created_at=1740512571, filename='train_data_one-prompt-response.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None, expires_at=None), FileObject(id='file-2xo2JhG1jiLt44PYw3JhWw', bytes=19404, created_at=1738869788, filename='step_metrics.csv', object='file', purpose='fine-tune-results', status='processed', status_details=None, expires_at=None), FileObject(id='file-DLgqcpU6cJ9zNsKiF5BQAy', bytes=122047, created_at=1738868724, filename='train

In [13]:
# Create fine-tuning job
client.fine_tuning.jobs.create(
    training_file="file-1Vp5noGdywWp1nD4U3enuY",
    model="gpt-4o-2024-08-06" # gpt-4o-mini-2024-07-18 or gpt-4o-2024-08-06
)

FineTuningJob(id='ftjob-vxxS7GRIaVhI4mgmIO7TNRby', created_at=1741736990, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-2024-08-06', object='fine_tuning.job', organization_id='org-RO3Int5VH4kDD0ib79PXZngn', result_files=[], seed=194789445, status='validating_files', trained_tokens=None, training_file='file-1Vp5noGdywWp1nD4U3enuY', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None, metadata=None, method={'type': 'supervised', 'supervised': {'hyperparameters': {'batch_size': 'auto', 'learning_rate_multiplier': 'auto', 'n_epochs': 'auto'}}})

In [41]:
# Check fine-tuning job
client.fine_tuning.jobs.list()
# or 
client.fine_tuning.jobs.retrieve("ftjob-vxxS7GRIaVhI4mgmIO7TNRby")

FineTuningJob(id='ftjob-vxxS7GRIaVhI4mgmIO7TNRby', created_at=1741736990, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-4o-2024-08-06', object='fine_tuning.job', organization_id='org-RO3Int5VH4kDD0ib79PXZngn', result_files=[], seed=194789445, status='queued', trained_tokens=None, training_file='file-1Vp5noGdywWp1nD4U3enuY', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None, metadata=None, method={'type': 'supervised', 'supervised': {'hyperparameters': {'n_epochs': 3, 'batch_size': 1, 'learning_rate_multiplier': 2.0}}})

In [53]:
client.fine_tuning.jobs.list_events(
  fine_tuning_job_id="ftjob-vxxS7GRIaVhI4mgmIO7TNRby",
  limit=1
)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-hupHo0H5UUE8eR4XeszV98Az', created_at=1741740001, level='info', message='The job has successfully completed', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=True)

In [ ]:
from openai import OpenAI
client = OpenAI(api_key="your_openai_api_key")

# Use fine-tuned model
completion = client.chat.completions.create(
    model="ft:gpt-4o-2024-08-06:personal::BA4QTgnH", # model id
    messages=[
        {"role": "user", "content": "Engine fire"}
    ]
)
print(f"The output of fine-tuned model is:\n{completion.choices[0].message.content}")

The output of fine-tuned model is:
Here's what to do if you detect an engine fire:

1. **Check for flames or smoke:** Monitor for any signs of an external fire.

2. **Mixture:** Set to idle.

3. **Fuel Selector:** Move to OFF.

4. **Magnetos:** Turn OFF.

5. **Ejector Door:** Open.

6. **Engine Out Procedure:** Follow the standard procedure for engine failure.

7. **Cranking:** Attempt to crank the engine to extinguish the fire.

8. **Power:** Set throttle to 1800 RPM, then shut down.

9. **Prepare to Evacuate:** Unlatch doors and prepare for a quick exit.

10. **Turn off:** All systems (standby, master, cabin heat/air, magnetos).

11. **Evacuate and Extinguish:** If there's visible fire, get everyone out and use an extinguisher.

12. **Complete shutdown:** After reaching 1800 RPM on a successful restart, power down the engine.
